Exercise 4: Design Theory
------------



## Activity 4.1: FDs & Closures
------

Recall that given a set of attributes  $\{A_1, \dots, A_n\}$ and a set of FDs $\Gamma$

The closure, denoted $\{A_1, \dots, A_n\}^+$, is defined to be the largest set of attributes B s.t. $$A_1,\dots,A_n \rightarrow B \text{ using } \Gamma.$$

We've built some functions to compute the closure of a set of attributes and other such operations (_feel free to look at the code- it's pretty simple and clean, if we say so ourselves..._):

In [ ]:
from closure import compute_closure

### Task 1 (25 pts)

Consider a schema with attributes $X=\{A,B,C,D,E,F,G,H\}$.

In this exercise, you are given a set of attributes $A\subset X$ and a set of FDs $F$.  Find **one FD** to add to $F$ so that the closure $A^+=X$

**Note: you can add FDs to the below set $F$ using e.g. `F.append((set([...]), set([...])))` and then check how you're doing using the `compute_closure` function from above!**

(As we'll find out immediately after this activity, this equivalent to saying: _Find one FD to add such that $A$ becomes a superkey for $X$_)

In [ ]:
A = set(['A', 'B','F'])
F = [(set(['A', 'C']), 'D'),
     (set(['D','H', 'G']), 'E'),
     (set(['A', 'B']), 'G'),
     (set(['F', 'B', 'G']), 'C')]

In [ ]:
compute_closure(A,F)

{'A', 'B', 'C', 'D', 'F', 'G'}

In [ ]:
F.append((set(['A']),'H'))
compute_closure(A, F)
# Explanation:
# At the first, we can't reach E and H.
# With the rule which we add, we can reach H.
# Also with using (set(['D','H', 'G']), 'E') dependency, we can reach E since we reached H with our new dependency.
# So, we can reach all of these there and A becomes superkey of X with that simple functional dependency.

{'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'}

## Activitity 4.2: Superkeys & Keys

Next, we'll add some new functions now for finding superkeys and keys.  Recall:
* A _superkey_ for a relation $R(B_1,\dots,B_m)$ is a set of attributes $\{A_1,\dots,A_n\}$ s.t.
$$ \{A_1,\dots,A_n\} \rightarrow B_{j} \text{ for all } j=1,\dots m$$
* A _key_ is a minimal (setwise) _superkey_

The algorithm to determine if a set of attributes $A$ is a superkey for $X$ is actually very simple- we just see if the $A^+=X$:

In [ ]:
def is_superkey_for(A, X, fds, verbose=False): 
    return X.issubset(compute_closure(A, fds, verbose=verbose))

Then, to check if $A$ is a key for $X$, we just confirm that:
* (a) it is a superkey
* (b) there are no smaller superkeys (_Note that we only need to check for superkeys of one size smaller- think about why!_)

In [ ]:
import itertools
def is_key_for(A, X, fds, verbose=False):
    subsets = set(itertools.combinations(A, len(A)-1))
    return is_superkey_for(A, X, fds) and \
        all([not is_superkey_for(set(SA), X, fds) for SA in subsets])

### Task 1 (10 pts)

Given the schema $R=\{A,B,C\}$, define a set of FDs such that there are **two- _and only two_- keys**, and check using the above functions!

In [ ]:
R = set(['A','B','C'])

In [ ]:
F = [(set(['A','B']), 'C'),
     (set(['A','C']), 'B')]

print(is_key_for(set(['A','B']), R, F)) # Key
print(is_key_for(set(['A','C']), R, F)) # Key
print(is_key_for(set(['B','C']), R, F)) # Not Key

True
True
False


### Task 2 (15 pts)

Now, given the below relation $R$, and the above tools, define a set of FDs to result in the most keys possible.  How many keys can you make?  Largest number wins it all!

_Bonus question: how many different sets of FDs will result in this maximum number of keys?_

In [ ]:
R = set(['A','B','C','D','E'])

In [ ]:
F = [(set(['A','B','C']), set(['D','E'])),
     (set(['A','B','D']), set(['C','E'])),
     (set(['A','B','E']), set(['C','D'])),
     (set(['A','C','D']), set(['B','E'])),
     (set(['A','C','E']), set(['B','D'])),
     (set(['A','D','E']), set(['B','C'])),
     (set(['B','C','D']), set(['A','E'])),
     (set(['B','C','E']), set(['A','D'])),
     (set(['B','D','E']), set(['A','C'])),
     (set(['C','D','E']), set(['A','B']))]
     
print(is_key_for(set(['A','B','C']), R, F))
print(is_key_for(set(['A','B','D']), R, F))
print(is_key_for(set(['A','B','E']), R, F))
print(is_key_for(set(['A','C','D']), R, F))
print(is_key_for(set(['A','C','E']), R, F))
print(is_key_for(set(['A','D','E']), R, F))
print(is_key_for(set(['B','C','D']), R, F))
print(is_key_for(set(['B','C','E']), R, F))
print(is_key_for(set(['B','D','E']), R, F))
print(is_key_for(set(['C','D','E']), R, F))

# Bonus question: How many different sets of FDs will result in this maximum number of keys?
# Answer: 
# C(5,0) = 1
# C(5,1) = 5
# C(5,2) = 10
# C(5,3) = 10
# C(5,4) = 5
# C(5,5) = 1
# C(5,2) = C(5,3) = 10. They will return maximum number of keys, so answer 2.

True
True
True
True
True
True
True
True
True
True


## Activity 4.3: BCNF Decompositions

The goal for this activity will be to compute some BCNF decompositions, using the tools from last lecture

First we'll load those tools, and some sample data:

In [ ]:
from closure import compute_closure, display_side_by_side, print_setup

In [ ]:
# make sure that sql is installed: pip install ipython-sql
%load_ext sql
%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @None'

In [ ]:
%%sql DROP TABLE IF EXISTS T;
CREATE TABLE T(course VARCHAR, classroom INT, time INT);
INSERT INTO T VALUES ('CS 364', 132, 900);
INSERT INTO T VALUES ('CS 245', 140, 1000);
INSERT INTO T VALUES ('EE 101', 210, 900);

 * sqlite://
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.


[]

### Task 1 (10 pts)

First, let's decompose `T` into BCNF!  Explicitly go through the steps of the BCNF algorithm using the `compute_closure` function, then decompose the following table (i.e. by creating new SQL tables) into BCNF:

We've also made a function, `display_side_by_side`, for nicer display!

In [ ]:
%sql SELECT * FROM T;

 * sqlite://
Done.


course,classroom,time
CS 364,132,900
CS 245,140,1000
EE 101,210,900


We are given the following FDs:

In [ ]:
A = set(['course', 'classroom', 'time'])
F = [('course', 'classroom'), (set(['classroom', 'time']), 'course')]
print_setup(A, F)

Attributes = {course,classroom,time}
FDs:

=> 	course -> classroom

=> 	classroom,time -> course


**Q:** What real-world constraints do these FDs express?

Now, use the `compute_closure` function to help decompose this table to BCNF:

In [ ]:
X = set(['course'])
compute_closure(X, F)

{'classroom', 'course'}

Compose into two tables, $T_1$ and $T_2$:

In [ ]:
%%sql
DROP TABLE IF EXISTS T1;
CREATE TABLE T1 AS SELECT DISTINCT * FROM (
  SELECT course, classroom FROM T
);

 * sqlite://
Done.
Done.


[]

In [ ]:
%%sql
DROP TABLE IF EXISTS T2;
CREATE TABLE T2 AS SELECT DISTINCT * FROM (
  SELECT course, time FROM T
);

 * sqlite://
Done.
Done.


[]

Now run the below to display the decomposed tables side-by-side:

In [ ]:
l = %sql SELECT * FROM T1;
r = %sql SELECT * FROM T2;
display_side_by_side(l,r)

# Answer 0f The Question Below:
# There are two conditions to be BCNF.
# 1) course → classroom (classroom ⊆ course)
# 2) Also course is superkey.
# It satisfies both of conditions, so yes it's a BCNF.

 * sqlite://
Done.
 * sqlite://
Done.


course classroom CS 364 132 CS 245 140 EE 101 210 course time CS 364 900 CS 245 1000 EE 101 900

**Q:** Is this now in BCNF?

-- Answer

### Task 2 (15 pts)

See if you can insert rows into $T_1$ and/or $T_2$ _which respect the local FDs that still hold_, such that **when $T_1$ and $T_2$ are now recomposed, the original FDs do not hold!**

In [ ]:
%%sql
INSERT INTO T1 VALUES ('CS 145', 132);
INSERT INTO T2 VALUES ('CS 145', 900);

 * sqlite://
1 rows affected.
1 rows affected.


[]

Now, reconstruct and print the re-composed table using a SQL query:

In [ ]:
# Fill _____
%%sql
SELECT T1.course, T1.classroom, T2.time
FROM T1, T2
WHERE T1.course = T2.course;

# Answer of The Question Below
# When tables are recomposed original functional dependencies don't hold!
# For example CS364 and CS145 have same classroom.

 * sqlite://
Done.


course,classroom,time
CS 364,132,900
CS 245,140,1000
EE 101,210,900
CS 145,132,900


**Q:** What went wrong?

-- Answer

## Activity 4.4 : MVDs

## Formal definition

Given a relation $R$ having attributes $A$, and two sets of attributes $X,Y\subseteq A$, the **_multi-value dependency (MVD)_** $X\twoheadrightarrow Y$ holds on $R$ if, for **any** tuples $t_1,t_2\in R$ s.t. $t_1[X] = t_2[X]$, there exists a tuple $t_3\in R$ s.t.:

* $t_3[X] = t_1[X] = t_2[X]$
* $t_3[Y] = t_1[Y]$
* $t_3[A\setminus Y] = t_2[A\setminus Y]$

where $A\setminus B$ = all elements of $A$ not in $B$.

So let's consider a toy example at this point:

In [ ]:
%%sql
DROP TABLE IF EXISTS R; CREATE TABLE R (A INT, B INT, C INT);
INSERT INTO R VALUES (1, 1, 0);
INSERT INTO R VALUES (1, 0, 1);
SELECT * FROM R;

 * sqlite://
Done.
Done.
1 rows affected.
1 rows affected.
Done.


A,B,C
1,1,0
1,0,1


Let's consider the first two rows as $t_1$ and $t_2$ respectively; what is the $t_3$ that the definition tells us we must add if we want the MVD $\{A\}\twoheadrightarrow\{B\}$ to hold?  Let's add it in:

In [ ]:
%sql INSERT INTO R VALUES (1,1,1); SELECT * FROM R;

 * sqlite://
1 rows affected.
Done.


A,B,C
1,1,0
1,0,1
1,1,1


However what about if we consider the first two rows as $t_2$ and $t_1$ respectively?  What row would the definition tell us we'd have to add in?

In [ ]:
%sql INSERT INTO R VALUES (1,0,0); SELECT * FROM R;

 * sqlite://
1 rows affected.
Done.


A,B,C
1,1,0
1,0,1
1,1,1
1,0,0


Now, if we checked every other pair like this, we'd see that we are done- the MVD $\{A\}\twoheadrightarrow\{B\}$ holds!

## A second definition

We see that this suggests another (equivalent) definition, which we'll phrase slightly less formally:

**For an MVD $\{A\}\twoheadrightarrow\{B\}$ to hold, for any pair of tuples that agree on attributes $A$, we also must find the corresponding _"swapped"_ pair: a pair of tuples that also have the same value of $A$ but have their $B$ and $(A\cup B)^C$ attributes swapped.**

(*where $(A\cup B)^C$ just means the attributes not in $A$ or $B$*)

This definition should make sense, and might even feel more intuitive, given the toy example above!

### Addressing one point of confusion:

Remember, an MVD holds over a **relation** (not a single tuple, not a single pair of tuples $t_1,t_2$...).  Again, look at the definition- an MVD, say $\{X\}\twoheadrightarrow\{Y\}$, holds over a relation $R$ if **_for any possible_** pair of tuples $t_1,t_2$ in $R$ such that $t_1[A]=t_2[A]$, the condition in the definition holds...


### Another clarification of terminology:
So how can we ever _test_ if an MVD holds over a relation $R$, based just on one _instance_ of $R$?  Couldn't someone always add in more tuples and violate it?  Aren't we being a little sloppy when we say an MVD 'holds' on an _instance_ of a relation just based on checking the instance..?

This is correct.  Recall that we have the same situation with FDs- we need to have _external information_ (such as a set of functional dependencies) to _prove_ that an MVD holds in general over a relation.  We **can** prove that it is violated or not violated by an instance of $R$, and that it thus could or could not potentially hold in general.

So when we say an MVD (or an FD) _holds_ on an instance, based only on checking the instance, we implictly mean the above- that based on the instance we see, it _could_ hold in general.

### A third definition...

At this point (especially if you're still reading) you may suspect a connection to cross-products and joins.  Consider decomposing our toy example in its **original state (before we added in the two rows to make the MVD hold)**.  Let's decompose it into two tables, split by the attribute sets of the MVD (recall it was $\{A\}\twoheadrightarrow\{B\}$):

In [ ]:
%%sql
DROP TABLE IF EXISTS R; CREATE TABLE R (A INT, B INT, C INT);
INSERT INTO R VALUES (1, 1, 0);
INSERT INTO R VALUES (1, 0, 1);
SELECT * FROM R;

 * sqlite://
Done.
Done.
1 rows affected.
1 rows affected.
Done.


A,B,C
1,1,0
1,0,1


In [ ]:
%sql DROP TABLE IF EXISTS R1; CREATE TABLE R1 AS SELECT A,B FROM R GROUP BY A,B;
%sql DROP TABLE IF EXISTS R2; CREATE TABLE R2 AS SELECT A,C FROM R GROUP BY A,C;
r1 = %sql SELECT * FROM R1;
r2 = %sql SELECT * FROM R2;
display_side_by_side(r1,r2)

 * sqlite://
Done.
Done.
 * sqlite://
Done.
Done.
 * sqlite://
Done.
 * sqlite://
Done.


A B 1 0 1 1 A C 1 0 1 1

Now let's take the join to recompose:

In [ ]:
%%sql
SELECT r1.A AS A, r1.B AS B, r2.C AS C
FROM R1 r1, R2 r2
WHERE r1.A = r2.A;

 * sqlite://
Done.


A,B,C
1,0,0
1,0,1
1,1,0
1,1,1


Woah!  The MVD that we decomposed along holds for the join of the decomposition!  We see that this is another definition for an MVD, in terms of the _'local joins'_: an MVD $\{A\}\twoheadrightarrow\{B\}$ holds if whenever we take a block of rows with the same value for the $A$ attributes, decompose it into $R_1(A,B)$ and $R_2(A,(A\cup B)^C)$, and then take the join of these tables, we end up with the same rows we had before!

### Task 1 (25 pts)

First load the following table:

In [ ]:
%%sql
DROP TABLE IF EXISTS courses;
CREATE TABLE courses (course TEXT, staff TEXT, student TEXT);
INSERT INTO courses VALUES ('CS949','Amy','Bob');
INSERT INTO courses VALUES ('CS145','Chris','Deb');
INSERT INTO courses VALUES ('CS145','Chris','Eli');
INSERT INTO courses VALUES ('CS145','Firas','Deb');
-- INSERT INTO courses VALUES ('CS145','Firas','Bob');
-- Firas Bob -> Firas (Firas Deb) + Bob (Amy Bob) -> MVD
INSERT INTO courses VALUES ('CS145','Firas','Eli');

 * sqlite://
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
Done.


[]

We developed some appealingly simple python tools for this lecture & last, but let's switch back to SQL quickly- write a query which returns **no values _only if_** the MVD course to staff holds

Then, comment out the insert statement(s) above so that the MVD does hold (do you remember how to comment out lines in SQL from earlier activities?  It's super useful!)

In [ ]:
%%sql
SELECT * FROM courses AS x, courses AS y
WHERE x.course = y.course AND NOT EXISTS (
    SELECT * FROM courses AS z
    WHERE z.course = x.course AND z.student = y.student AND z.staff = x.staff);

 * sqlite://
Done.


course,staff,student,course_1,staff_1,student_1
